In [38]:
import requests
import numpy as np
from bs4 import BeautifulSoup
import selenium
from selenium.webdriver import Firefox
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep

/html/body/div[3]/table/tbody/tr/td/table[1]/tbody/tr/td[2]/form/table/tbody/tr[2]/td[1]/table/tbody/tr/td/div[6]/table/tbody/tr/td[13]/a

In [39]:
def get_articles_adress(page):
    '''
    Возвращает список адресов всех статей
    '''
    articles_adrees = []
    id_str = '/item.asp?id='
    for i in page.findAll('a'):
        try:
            if id_str in i['href']:
                num_id = i['href']
                articles_adrees.append(https + num_id)
        except:
            continue
    return articles_adrees

def find_data(adress):
    '''
    Возвращает list из названия [статьи, авторы, абстракт]
    '''
    assert isinstance(adress, str)
    sleep(np.random.rand() * 2)
    art = requests.get(adress)
    soup = BeautifulSoup(art.content.decode('utf-8'), "html.parser")
    data = []
    for i in soup.findAll('input'):
        try:    
            if not i['value'][0].isdigit() and (i['value'][0] == i['value'][0].upper()):
                data.append(i['value'])
        except:
            continue
    data.append(soup.find('p').text)
    return data


/html/body/div[3]/table/tbody/tr/td/table[1]/tbody/tr/td[2]/form/table/tbody/tr[2]/td[1]/table/tbody/tr/td/div[6]/table/tbody/tr/td[13]/a


In [60]:
https = "https://elibrary.ru"
artical_data = {}
artical_adresses = []
xp_main    = '/html/body/div[3]/table/tbody/tr/td/table[1]/tbody/tr/td[2]/form/table/tbody/tr[2]/td[1]/table/tbody/tr/td/div[6]/table/tbody/tr/td[13]/a'
xp_reserve = '/html/body/div[2]/table/tbody/tr/td/table[1]/tbody/tr/td[2]/form/table/tbody/tr[2]/td[1]/table/tbody/tr/td/div[6]/table/tbody/tr/td[13]/a'

browser = Firefox(executable_path="./geckodriver") # ./geckodriver тоже вроде можно
wait = WebDriverWait(browser, 30, 1)
browser.get('https://elibrary.ru/title_items.asp?id=21813')                          # Заходим на первую страницу журнала
for i in range(4):
    search_form = wait.until(EC.visibility_of_element_located((By.XPATH, xp_main)))
    page = BeautifulSoup(browser.page_source, "html.parser")                             # Забираем страничку и преобразуем ее в BS
    new_id = get_articles_adress(page)                                                   # Собираем со странички все адреса статей
    artical_adresses.append(new_id)                                                      # Добавляем адреса в хранилище
    search_form.click()                                                                  # Жмякаем на кнопку


In [61]:
artical_adresses[0] == artical_adresses[1]

False

In [62]:
artical_adresses

[['https://elibrary.ru/item.asp?id=30076781',
  'https://elibrary.ru/item.asp?id=29640350',
  'https://elibrary.ru/item.asp?id=29628631',
  'https://elibrary.ru/item.asp?id=29607996',
  'https://elibrary.ru/item.asp?id=29629225',
  'https://elibrary.ru/item.asp?id=29589627',
  'https://elibrary.ru/item.asp?id=29622770',
  'https://elibrary.ru/item.asp?id=29613318',
  'https://elibrary.ru/item.asp?id=29644770',
  'https://elibrary.ru/item.asp?id=29644751',
  'https://elibrary.ru/item.asp?id=29700062',
  'https://elibrary.ru/item.asp?id=29632444',
  'https://elibrary.ru/item.asp?id=29646031',
  'https://elibrary.ru/item.asp?id=29646026',
  'https://elibrary.ru/item.asp?id=29630080',
  'https://elibrary.ru/item.asp?id=29638928',
  'https://elibrary.ru/item.asp?id=29694116',
  'https://elibrary.ru/item.asp?id=29489054',
  'https://elibrary.ru/item.asp?id=29634570',
  'https://elibrary.ru/item.asp?id=29640640'],
 ['https://elibrary.ru/item.asp?id=29640291',
  'https://elibrary.ru/item.asp?i

In [63]:
len(artical_adresses)

4

In [12]:
xp  = '/html/body/div[3]/table/tbody/tr/td/table[1]/tbody/tr/td[2]/form/table/tbody/tr[2]/td[1]/table/tbody/tr/td/div[6]/table/tbody/tr/td[13]/a'
xp1 = '/html/body/div[2]/table/tbody/tr/td/table[1]/tbody/tr/td[2]/form/table/tbody/tr[2]/td[1]/table/tbody/tr/td/div[6]/table/tbody/tr/td[13]/a'
xp2 = '/html/body/div[3]/table/tbody/tr/td/table[1]/tbody/tr/td[2]/form/table/tbody/tr[2]/td[1]/table/tbody/tr/td/div[6]/table/tbody/tr/td[13]/a'
xp3 = '/html/body/div[3]/table/tbody/tr/td/table[1]/tbody/tr/td[2]/form/table/tbody/tr[2]/td[1]/table/tbody/tr/td/div[6]/table/tbody/tr/td[13]/a'
xp4 = '/html/body/div[3]/table/tbody/tr/td/table[1]/tbody/tr/td[2]/form/table/tbody/tr[2]/td[1]/table/tbody/tr/td/div[6]/table/tbody/tr/td[13]/a'
xp5 = '/html/body/div[3]/table/tbody/tr/td/table[1]/tbody/tr/td[2]/form/table/tbody/tr[2]/td[1]/table/tbody/tr/td/div[6]/table/tbody/tr/td[13]/a'
xp6 = '/html/body/div[3]/table/tbody/tr/td/table[1]/tbody/tr/td[2]/form/table/tbody/tr[2]/td[1]/table/tbody/tr/td/div[6]/table/tbody/tr/td[13]/a'
xp7 = '/html/body/div[3]/table/tbody/tr/td/table[1]/tbody/tr/td[2]/form/table/tbody/tr[2]/td[1]/table/tbody/tr/td/div[6]/table/tbody/tr/td[13]/a'
xp8 = '/html/body/div[3]/table/tbody/tr/td/table[1]/tbody/tr/td[2]/form/table/tbody/tr[2]/td[1]/table/tbody/tr/td/div[6]/table/tbody/tr/td[13]/a'

In [13]:
xp8 == xp

True

In [66]:

print('I found next id: ', new_id)
#     except:
#         print('Just forget it')
# #     for adress in artical_adresses:
# #         d = find_data(adress)
# #         artical_data[d[1]] = [d[0], d[2]]
    

Press:  /html/body/div[3]/table/tbody/tr/td/table[1]/tbody/tr/td[2]/form/table/tbody/tr[2]/td[1]/table/tbody/tr/td/div[6]/table/tbody/tr/td[13]/a
I found next id:  ['https://elibrary.ru/item.asp?id=29601180', 'https://elibrary.ru/item.asp?id=29578204', 'https://elibrary.ru/item.asp?id=29604436', 'https://elibrary.ru/item.asp?id=29604192', 'https://elibrary.ru/item.asp?id=29630077', 'https://elibrary.ru/item.asp?id=29578370', 'https://elibrary.ru/item.asp?id=29558918', 'https://elibrary.ru/item.asp?id=29586568', 'https://elibrary.ru/item.asp?id=29558963', 'https://elibrary.ru/item.asp?id=29630375', 'https://elibrary.ru/item.asp?id=29586795', 'https://elibrary.ru/item.asp?id=29611932', 'https://elibrary.ru/item.asp?id=29615458', 'https://elibrary.ru/item.asp?id=29619415', 'https://elibrary.ru/item.asp?id=29598136', 'https://elibrary.ru/item.asp?id=29565977', 'https://elibrary.ru/item.asp?id=29638232', 'https://elibrary.ru/item.asp?id=29592135', 'https://elibrary.ru/item.asp?id=29592355',

In [32]:
for page_num in range(4, 7):
    sleep(2 * np.random.random())
    xp = '/html/body/div[3]/table/tbody/tr/td/table[1]/tbody/tr/td[2]/form/table/tbody/tr[2]/td[1]/table/tbody/tr/td/div[6]/table/tbody/tr/td[' + str(page_num+1) + ']/a'
    try:
        print("Press: ", xp)
        search_form = browser.find_element_by_xpath(xp)
        search_form.send_keys(Keys.ENTER)
        print("I'm on page {}".format(page_num-2))
        page = BeautifulSoup(browser.page_source, "html.parser")
        new_id = get_articles_adress(page)
        artical_adresses.append(new_id)
        print('I found next id: ', new_id)
    except:
        print('Just forget it')
#     for adress in artical_adresses:
#         d = find_data(adress)
#         artical_data[d[1]] = [d[0], d[2]]
    

I found next id:  ['https://elibrary.ru/item.asp?id=29621052', 'https://elibrary.ru/item.asp?id=29569074', 'https://elibrary.ru/item.asp?id=29613213', 'https://elibrary.ru/item.asp?id=29599695', 'https://elibrary.ru/item.asp?id=29639343', 'https://elibrary.ru/item.asp?id=29607042', 'https://elibrary.ru/item.asp?id=29643122', 'https://elibrary.ru/item.asp?id=29700189', 'https://elibrary.ru/item.asp?id=29478497', 'https://elibrary.ru/item.asp?id=29631769', 'https://elibrary.ru/item.asp?id=29579757', 'https://elibrary.ru/item.asp?id=29631798', 'https://elibrary.ru/item.asp?id=29689105', 'https://elibrary.ru/item.asp?id=29593485', 'https://elibrary.ru/item.asp?id=29639238', 'https://elibrary.ru/item.asp?id=29621076', 'https://elibrary.ru/item.asp?id=29600301', 'https://elibrary.ru/item.asp?id=29625780', 'https://elibrary.ru/item.asp?id=29601471', 'https://elibrary.ru/item.asp?id=29594806']
Press:  /html/body/div[3]/table/tbody/tr/td/table[1]/tbody/tr/td[2]/form/table/tbody/tr[2]/td[1]/tabl

In [28]:
type(browser.page_source)

str

In [22]:
type(page)

bs4.BeautifulSoup

In [29]:
artical_adresses

[['https://elibrary.ru/item.asp?id=29621052',
  'https://elibrary.ru/item.asp?id=29569074',
  'https://elibrary.ru/item.asp?id=29613213',
  'https://elibrary.ru/item.asp?id=29599695',
  'https://elibrary.ru/item.asp?id=29639343',
  'https://elibrary.ru/item.asp?id=29607042',
  'https://elibrary.ru/item.asp?id=29643122',
  'https://elibrary.ru/item.asp?id=29700189',
  'https://elibrary.ru/item.asp?id=29478497',
  'https://elibrary.ru/item.asp?id=29631769',
  'https://elibrary.ru/item.asp?id=29579757',
  'https://elibrary.ru/item.asp?id=29631798',
  'https://elibrary.ru/item.asp?id=29689105',
  'https://elibrary.ru/item.asp?id=29593485',
  'https://elibrary.ru/item.asp?id=29639238',
  'https://elibrary.ru/item.asp?id=29621076',
  'https://elibrary.ru/item.asp?id=29600301',
  'https://elibrary.ru/item.asp?id=29625780',
  'https://elibrary.ru/item.asp?id=29601471',
  'https://elibrary.ru/item.asp?id=29594806'],
 [],
 ['https://elibrary.ru/item.asp?id=29621052',
  'https://elibrary.ru/item.

In [8]:
len(artical_data)

20